# 📘 P1.2.4.5 – Python Asynchronous Programming
## Topic: Error Handling in Async Code

## Real time Scenario

You run three async tasks at once:
- Load user profile
- Load messages
- Load notifications

If one task fails, you still want the others to finish and show what worked.

In this notebook, you'll learn to:
1. Catch exceptions in async functions
2. Handle errors when using `gather()`
3. Set timeouts to avoid hangs
4. Retry simple failures

## Why Error Handling Matters

In async code, several tasks run at the same time.
If one fails, you must decide:
- Should the others keep running?
- Should you show partial results?
- Should you retry?

**Goal**: keep your app stable and still return useful results.

## Pattern 1: Try-Except in Async Functions

Wrap `await` calls in try-except blocks, just like synchronous code:

In [ ]:
import asyncio

async def risky_operation(operation_id, should_fail=False):
    """Simulate an operation that might fail"""
    try:
        print(f"🔄 Operation {operation_id} starting...")
        await asyncio.sleep(0.2)
        
        if should_fail:
            raise ValueError(f"Operation {operation_id} failed intentionally")
        
        print(f"✅ Operation {operation_id} succeeded")
        return f"Result {operation_id}"
    
    except ValueError as e:
        print(f"❌ Error in operation {operation_id}: {e}")
        return None  # Return None to indicate failure

async def main():
    result1 = await risky_operation(1, should_fail=False)
    result2 = await risky_operation(2, should_fail=True)
    result3 = await risky_operation(3, should_fail=False)
    
    print(f"\nResults: {result1}, {result2}, {result3}")

await main()

## Pattern 2: gather() with return_exceptions=True

When running multiple tasks concurrently, use `return_exceptions=True` to catch errors without crashing:

In [ ]:
async def fetch_data(source, delay, should_fail=False):
    """Fetch data from a source"""
    print(f"📡 Fetching from {source}...")
    await asyncio.sleep(delay)
    
    if should_fail:
        raise ConnectionError(f"{source} is unreachable")
    
    return f"{source}: 100 records"

async def collect_data():
    """Fetch from multiple sources, handling failures gracefully"""
    tasks = [
        fetch_data("Database A", 0.3, should_fail=False),
        fetch_data("Database B", 0.3, should_fail=True),   # This will fail
        fetch_data("Database C", 0.3, should_fail=False),
    ]
    
    # return_exceptions=True: Returns exceptions as values instead of raising
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    print("\n📊 Results:")
    for i, result in enumerate(results, 1):
        if isinstance(result, Exception):
            print(f"  Source {i}: ❌ {type(result).__name__}: {result}")
        else:
            print(f"  Source {i}: ✅ {result}")
    
    return results

await collect_data()

## Pattern 3: Timeout Exceptions

Handle timeouts specifically when requests take too long:

In [ ]:
async def slow_service(name, delay):
    """Simulate a service with variable response time"""
    print(f"🌐 {name} service started")
    await asyncio.sleep(delay)
    print(f"🌐 {name} service finished")
    return f"{name} response"

async def fetch_with_timeout_handling():
    """Fetch with timeout and handle TimeoutError specifically"""
    timeout = 0.5  # 500ms timeout
    
    tasks = [
        asyncio.wait_for(slow_service("Fast", 0.2), timeout=timeout),
        asyncio.wait_for(slow_service("Slow", 1.5), timeout=timeout),  # Will timeout
        asyncio.wait_for(slow_service("Medium", 0.4), timeout=timeout),
    ]
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    print("\n⏱️  Timeout Results:")
    for i, result in enumerate(results, 1):
        if isinstance(result, asyncio.TimeoutError):
            print(f"  Task {i}: ⏱️  TIMEOUT - Request exceeded deadline")
        elif isinstance(result, Exception):
            print(f"  Task {i}: ❌ {type(result).__name__}")
        else:
            print(f"  Task {i}: ✅ {result}")

await fetch_with_timeout_handling()

## ⚠️ Common Mistakes

### Mistake 1: Forgetting `return_exceptions=True`

```python
# ❌ WRONG - one error stops everything
results = await asyncio.gather(task1, task2, task3)

# ✅ CORRECT - handle errors after
results = await asyncio.gather(task1, task2, task3, return_exceptions=True)
```

### Mistake 2: No timeout for slow tasks

```python
# ❌ WRONG - could hang forever
result = await fetch_data()

# ✅ CORRECT - add a timeout
result = await asyncio.wait_for(fetch_data(), timeout=2.0)
```

## 🎯 Key Takeaways

1. Use `try/except` around async work just like sync code
2. Prefer `gather(..., return_exceptions=True)` when running many tasks
3. Add timeouts to avoid hanging forever
5. Handle partial results instead of crashing